# SSCx connectome rewiring based on simplified models of connectivity
This is an auxiliary notebook for configuring 1st..5th-order connectome rewiring, which
 - sets up connectome manipulations (rewiring)
 - writes .json config file for running rewiring using <code>sbatch</code>
 - allows iterative matching of overall numbers of connections

ℹ️ Part of reproduction of simplified connectomes experiment described in [Pokorny et al. (2024)](https://doi.org/10.1101/2024.05.24.593860)

<u>Requirements</u>:
- [Connectome-Manipulator](https://github.com/BlueBrain/connectome-manipulator) (Python venv)
- [SSCx network model](https://doi.org/10.5281/zenodo.8026353)
- Fitted stochastic and auxiliary models, see `SSCx_model_fitting.ipynb` (also available in Zenodo dataset)

_SSCx network model:_ Download and extract `O1_data_physiology_withfix.xz` and set path accordingly:

In [1]:
# Path to SSCx network model
circuit_config = '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data/circuit_config.json'

In [18]:
# Initialization

""" Global imports """
import json
import numpy as np
import os
import tqdm

In [19]:
def default_manip_config(circuit_config, seed=3210, N_split=None):
    """Generates a default manipulation config dict w/o any specific manipulation."""
    manip_config = {}
    manip_config['circuit_config'] = circuit_config
    manip_config['seed'] = seed
    if N_split is not None:
        manip_config['N_split_nodes'] = N_split
    return manip_config

def order_from_name(name):
    order_str = name.split("Order")[0]
    order = [int(i) for i in order_str if i.isdigit()]
    assert len(order) == 1, "ERROR: Order could not be determined!"
    return order[0]

def order_from_model_file(model_file):
    with open(model_file, "r") as f:
        model_dict = json.load(f)
    order = order_from_name(model_dict.get("model", ""))
    assert order > 0, "ERROR: Invalid model order!"
    return order

def add_manip_to_config(manip_config, pos_map_file, prob_model_file, delay_model_file, props_model_file, calib_run=True, p_scale=None):
    """Adds a specific n-th order rewiring operation to the (v4) manipulation config dict (in-place)."""
    assert 'manip' not in manip_config, "ERROR: Manipulation operation already specified!"
    order = order_from_model_file(prob_model_file)
    if p_scale is None:
        p_scale = 1.0
    else:
        assert p_scale > 0.0, "ERROR: p_scale out of range!"
    manip_config['manip'] = {'name': f'ConnRewireOrder{order}Hex0EE',
                             'fcts': [{'source': 'conn_rewiring',
                                       'sel_src': {'node_set': 'hex0', 'synapse_class': 'EXC'},
                                       'sel_dest': {'node_set': 'hex0', 'synapse_class': 'EXC'},
                                       'syn_class': 'EXC',
                                       'keep_indegree': False,
                                       'reuse_conns': False,
                                       'keep_conns': False,
                                       'gen_method': 'randomize',
                                       'syn_pos_mode': 'reuse',
                                       'amount_pct': 100,
                                       'estimation_run': calib_run,
                                       'opt_nconn': not calib_run,
                                       'p_scale': p_scale,
                                       'pos_map_file': pos_map_file,
                                       'model_config': {'prob_model_spec': {'file': prob_model_file},
                                                        'delay_model_spec': {'file': delay_model_file},
                                                        'props_model_spec': {'file': props_model_file}}}]}

def export_manip_config(manip_config, config_path, print_cmd=False, circuit_name=None, output_base_path=None, N_parallel=None, calib_run=False):
    """Writes manipulation config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    fn = f'manip_config__{manip_config["manip"]["name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(manip_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

    if print_cmd:
        assert circuit_name is not None and output_base_path is not None and N_parallel is not None, \
            "ERROR: circuit_name/output_base_path/N_parallel required for printing launch command!"
        print()
        output_dir = os.path.join(output_base_path, circuit_name + f'__{manip_config["manip"]["name"]}')
        print_launch_cmd(cpath, fn, output_dir, N_parallel, calib_run)

def print_launch_cmd(config_path, config_fn, output_dir, N_parallel, calib_run):
    if calib_run:
        run_cmd = f'sbatch run_rewiring_parallel__estimation_run.sh "{config_fn}" "{output_dir}" {N_parallel}'
    else:
        run_cmd = f'sbatch run_rewiring_parallel.sh "{config_fn}" "{output_dir}" {N_parallel}'
    print(f"# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]")
    print(f"cd {config_path}")
    print(run_cmd)


ℹ️ Configure config, model, and output paths:

In [14]:
# Rewiring configuration
circuit_name = 'SSCx-HexO1-Release'
output_base_path = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/Zenodo/SSCx-connectome-manipulation-data/simplified_connectomes/circuits'
config_path = '../configs'  # Must exist!
models_path = os.path.join('/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/Zenodo/SSCx-connectome-manipulation-data/simplified_connectomes/model_building', circuit_name, 'model')  # Location of model files

In [6]:
# Model selection (hex0 column models!!)
flatpos_model_file = os.path.join(models_path, 'FlatPosMapping-SSCxO1.json')
prob_model_files = [os.path.join(models_path, 'ConnProb1stOrder-SSCxO1-Hex0EE.json'),
                    os.path.join(models_path, 'ConnProb2ndOrderComplex-SSCxO1-Hex0EE.json'),
                    os.path.join(models_path, 'ConnProb3rdOrderComplex-SSCxO1-Hex0EE.json'),
                    os.path.join(models_path, 'ConnProb4thOrder-SSCxO1-Hex0EE.json'),
                    os.path.join(models_path, 'ConnProb5thOrder-SSCxO1-Hex0EE.json')]
delay_model_file = os.path.join(models_path, 'DistDepDelay-SSCxO1-Hex0EE.json')
props_model_file = os.path.join(models_path, 'ConnPropsPerPathway-SSCxO1-Hex0EE.json')


## Create rewiring configs and run rewiring (estimation)
- Central column (hex0) of SSCx-HexO1 connectome
- EXC-EXC connections only

ℹ️ To run rewiring, configure the SLURM script `run_rewiring_parallel<__estimation_run>.sh` according to the used computation system and run below LAUNCH COMMANDS for generating the rewired circuits.

ℹ️ All rewired circuits are also contained in the Zenodo dataset

IMPORTANT: Folloing steps are required for matching the overall numbers of connections for a given Nth-order model:
1. Generate `manip_config` with "estimation run" option (`calib_run=True`)
2. Launch rewiring
3. Extract connection counts from data logs (follow section "Matching number of connections" below)
4. Update `p_scale` and repeat from step 1. (until converged or oscillating)
5. Finally, generate `manip_config` without "estimation run" option (`calib_run=False`) and launch rewiring, which will produce the actual connectome

In [20]:
# 1st-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)

# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-0__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 0.9996751184443766)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-1__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 0.9996751184443766 * 0.9996751184443766)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-2__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-3__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-4__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-5__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766 * 0.9996751184443766 * 1.003405249483348)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-6__'
# export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500, calib_run=True)

add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[0], delay_model_file, props_model_file, calib_run=False, p_scale=1.0)  # OPTIMUM
export_manip_config(manip_config, config_path, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder1Hex0EE.json written to ../configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd ../configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder1Hex0EE.json" "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/circuits_v2/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE" 500


In [21]:
# 2nd-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)

# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-0__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-1__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405 * 0.9999923647346808)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-2__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405 * 0.9999923647346808 * 1.0000015270670553)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-3__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405 * 0.9999923647346808 * 1.0000015270670553 * 1.0000004164723708)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-4__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405 * 0.9999923647346808 * 1.0000015270670553 * 1.0000004164723708 * 1.0000001388240851)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-5__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405 * 0.9999923647346808 * 1.0000015270670553 * 1.0000004164723708 * 1.0000001388240851 * 0.9999997223519456)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-6__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0044481864367405 * 0.9999923647346808 * 1.0000015270670553 * 1.0000004164723708 * 1.0000001388240851 * 0.9999997223519456 * 1.0000002776482086)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-7__'
# export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500, calib_run=True)

add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[1], delay_model_file, props_model_file, calib_run=False, p_scale=1.0044424693787228)  # OPTIMUM
export_manip_config(manip_config, config_path, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder2Hex0EE.json written to ../configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd ../configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder2Hex0EE.json" "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/circuits_v2/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE" 500


In [22]:
# 3rd-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)

# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[2], delay_model_file, props_model_file, calib_run=True, p_scale=1.0)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-0__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[2], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.004624555260151)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-1__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[2], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.004624555260151 * 1.000005275342328)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-2__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[2], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.004624555260151 * 1.000005275342328 * 1.0000005552965714)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-3__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[2], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.004624555260151 * 1.000005275342328 * 1.0000005552965714 * 0.9999997223519456)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-4__'
# export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500, calib_run=True)

add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[2], delay_model_file, props_model_file, calib_run=False, p_scale=1.004630412866105)  # OPTIMUM
export_manip_config(manip_config, config_path, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder3Hex0EE.json written to ../configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd ../configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder3Hex0EE.json" "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/circuits_v2/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE" 500


In [23]:
# 4th-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)

# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=True, p_scale=1.0)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-0__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0207829350335418)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-1__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0207829350335418 * 1.0000151320521438)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-2__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0207829350335418 * 1.0000151320521438 * 0.9999987505849692)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-3__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0207829350335418 * 1.0000151320521438 * 0.9999987505849692 * 1.0000001388240851)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-4__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0207829350335418 * 1.0000151320521438 * 0.9999987505849692 * 1.0000001388240851 * 1.0000001388240851)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-5__'
# export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500, calib_run=True)

add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[3], delay_model_file, props_model_file, calib_run=False, p_scale=1.0207973895957692)  # OPTIMUM
export_manip_config(manip_config, config_path, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder4Hex0EE.json written to ../configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd ../configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder4Hex0EE.json" "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/circuits_v2/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE" 500


In [24]:
# 5th-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)

# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[4], delay_model_file, props_model_file, calib_run=True, p_scale=1.0)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-0__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[4], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0195708704147093)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-1__'
# add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[4], delay_model_file, props_model_file, calib_run=True, p_scale=1.0 * 1.0195708704147093 * 0.9999798709156527)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-2__'
# export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500, calib_run=True)

add_manip_to_config(manip_config, flatpos_model_file, prob_model_files[4], delay_model_file, props_model_file, calib_run=False, p_scale=1.0195503473866605)  # OPTIMUM
export_manip_config(manip_config, config_path, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder5Hex0EE.json written to ../configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd ../configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder5Hex0EE.json" "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/circuits_v2/SSCx-HexO1-Release__ConnRewireOrder5Hex0EE" 500


---
## Matching number of connections

<u>Procedure</u>:
1. Initialize scaling factor `p_scale = 1.0`
2. Run calibration (evaulation) run with `p_scale`
3. Check connection counts from data logs and compute new scaling factor `p_scale'`
4. Adjust scaling factor `p_scale = p_scale * p_scale'`
5. Repeat steps 2. to 5. until convergence$^1$ (or oscillation)
6. Run actual rewiring with converged `p_scale` (or closest value, in case of oscillation)

$^1$Convergence ... Number of connections match exactly

### Check connection counts and adjust scaling factor

ℹ️ Set log type and file path below pointing to the main log file of the respective estimation run

In [71]:
# Estimate global probability scaling factor for matching #connections (from log file of estimation or actual rewiring run)
log_type = 'Rewiring' # 'Rewiring'...actual rewiring, 'Estimation'...estimation run
N_split = 500

# 1st-order rewiring
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-0__/logs/connectome_manipulation_2023-08-25_12h24.log'  # DIFF = 2341 (0.03% rel. to wiring target) => OPTIMUM
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-1__/logs/connectome_manipulation_2023-08-25_14h01.log'  # DIFF = 2341 (0.03% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-2__/logs/connectome_manipulation_2023-08-25_14h21.log'  # DIFF = 2341 (0.03% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-3__/logs/connectome_manipulation_2023-08-25_14h52.log'  # DIFF = 2341 (0.03% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-4__/logs/connectome_manipulation_2023-08-25_15h02.log'  # DIFF = 2341 (0.03% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-5__/logs/connectome_manipulation_2023-08-25_16h04.log'  # DIFF = -24446 (-0.34% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE__calib-6__/logs/connectome_manipulation_2023-08-25_16h23.log'  # DIFF = 2341 (0.03% rel. to wiring target) => OSCILLATING
# p_scales = [1.0, 0.9996751184443766, 0.9996751184443766, 0.9996751184443766, 0.9996751184443766, 0.9996751184443766, 1.003405249483348, 0.9996751184443766]
# p_scale_opt = np.prod(p_scales[:1]) # = 1.0
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder1Hex0EE/logs/connectome_manipulation_2023-08-25_18h01.log'  # DIFF = 2341 (0.03% rel. to wiring target) => ACTUAL REWIRING RUN

# 2nd-order rewiring
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-0__/logs/connectome_manipulation_2023-08-25_15h55.log'  # DIFF = -31900 (-0.44% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-1__/logs/connectome_manipulation_2023-08-25_16h13.log'  # DIFF = 55 (0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-2__/logs/connectome_manipulation_2023-08-25_16h25.log'  # DIFF = -11 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-3__/logs/connectome_manipulation_2023-08-25_16h38.log'  # DIFF = -3 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-4__/logs/connectome_manipulation_2023-08-25_17h59.log'  # DIFF = -1 (-0.00% rel. to wiring target) => OPTIMUM
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-5__/logs/connectome_manipulation_2023-08-28_10h08.log'  # DIFF = 2 (0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-6__/logs/connectome_manipulation_2023-08-28_10h15.log'  # DIFF = -2 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE__calib-7__/logs/connectome_manipulation_2023-08-28_10h21.log'  # DIFF = 2 (0.00% rel. to wiring target) => OSCILLATING
# p_scales = [1.0, 1.0044481864367405, 0.9999923647346808, 1.0000015270670553, 1.0000004164723708, 1.0000001388240851, 0.9999997223519456, 1.0000002776482086, 0.9999997223519456]
# p_scale_opt = np.prod(p_scales[:5]) # = 1.0044424693787228
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder2Hex0EE/logs/connectome_manipulation_2023-08-28_10h28.log'  # DIFF = -1 (-0.00% rel. to wiring target) => ACTUAL REWIRING RUN

# 3rd-order rewiring
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE__calib-0__/logs/connectome_manipulation_2023-08-28_10h22.log'  # DIFF = -33159 (-0.46% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE__calib-1__/logs/connectome_manipulation_2023-08-28_10h36.log'  # DIFF = -38 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE__calib-2__/logs/connectome_manipulation_2023-08-28_10h40.log'  # DIFF = -4 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE__calib-3__/logs/connectome_manipulation_2023-08-28_10h46.log'  # DIFF = 2 (0.00% rel. to wiring target) => OPTIMUM
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE__calib-4__/logs/connectome_manipulation_2023-08-28_10h51.log'  # DIFF = -4 (-0.00% rel. to wiring target) => OSCILLATING
# p_scales = [1.0, 1.004624555260151, 1.000005275342328, 1.0000005552965714, 0.9999997223519456, 1.0000005552965714]
# p_scale_opt = np.prod(p_scales[:4]) # = 1.004630412866105
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder3Hex0EE/logs/connectome_manipulation_2023-08-28_10h56.log'  # DIFF = 2 (0.00% rel. to wiring target) => ACTUAL REWIRING RUN

# 4th-order rewiring
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE__calib-0__/logs/connectome_manipulation_2023-08-28_10h41.log'  # DIFF = -146659 (-2.04% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE__calib-1__/logs/connectome_manipulation_2023-08-28_10h48.log'  # DIFF = -109 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE__calib-2__/logs/connectome_manipulation_2023-08-28_10h52.log'  # DIFF = 9 (0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE__calib-3__/logs/connectome_manipulation_2023-08-28_10h58.log'  # DIFF = -1 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE__calib-4__/logs/connectome_manipulation_2023-08-28_11h04.log'  # DIFF = -1 (-0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE__calib-5__/logs/connectome_manipulation_2023-08-28_11h11.log'  # DIFF = 0 (0.00% rel. to wiring target) => OPTIMUM (CONVERGED)
# p_scales = [1.0, 1.0207829350335418, 1.0000151320521438, 0.9999987505849692, 1.0000001388240851, 1.0000001388240851, 1.0]
# p_scale_opt = np.prod(p_scales[:6]) # = 1.0207973895957692
main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder4Hex0EE/logs/connectome_manipulation_2023-08-28_11h21.log'  # DIFF = 0 (0.00% rel. to wiring target) => ACTUAL REWIRING RUN

# 5th-order rewiring
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder5Hex0EE__calib-0__/logs/connectome_manipulation_2023-08-28_10h41.log'  # DIFF = -138270 (-1.92% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder5Hex0EE__calib-1__/logs/connectome_manipulation_2023-08-28_10h49.log'  # DIFF = 145 (0.00% rel. to wiring target)
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder5Hex0EE__calib-2__/logs/connectome_manipulation_2023-08-28_10h54.log'  # DIFF = 0 (0.00% rel. to wiring target) => OPTIMUM (CONVERGED)
# p_scales = [1.0, 1.0195708704147093, 0.9999798709156527, 1.0]
# p_scale_opt = np.prod(p_scales[:3]) # = 1.0195503473866605
# main_log_file = f'{output_base_path}/SSCx-HexO1-Release__ConnRewireOrder5Hex0EE/logs/connectome_manipulation_2023-08-28_11h02.log'  # DIFF = xxx (xxx% rel. to wiring target) => ACTUAL REWIRING RUN

log_path, log_fn = os.path.split(main_log_file)  # Main log path/filename
log_date = os.path.splitext(log_fn)[0].split('connectome_manipulation_')[1]

log_folders = [os.path.join(log_path, d) for d in os.listdir(log_path) if os.path.isdir(os.path.join(log_path, d))]
if log_type == 'Rewiring':  # In rewiring run, an ..Indices.. data log must always exist for all splits (so we can check for missing splits)
    data_log_splits = [f'{log_date}.{log_type}Indices_{i + 1}_{N_split}.npz' for i in range(N_split)]
else:  # ... whereas in estimation run, a ..Stats.. data log must always exist for all splits (so we can check for missing splits)
    data_log_splits = [f'{log_date}.{log_type}Stats_{i + 1}_{N_split}.npz' for i in range(N_split)]

conn_count_orig_all = 0 # Overall input connection count
conn_count_pred_all = 0 # Overall output connection count after rewiring (average prediction)
conn_count_actu_all = 0 # Overall output connection count after rewiring (actual value)
conn_count_orig_sel = 0 # Input connection count within selected rewiring target
conn_count_pred_sel = 0 # Output connection count within selected rewiring target after rewiring (average prediction)
conn_count_actu_sel = 0 # Output connection count within selected rewiring target after rewiring (actual value)
diff_from_avg = []
split_count = 0
for split_name in tqdm.tqdm(data_log_splits, desc="Collecting log files"):
    # Search for log file in all log folders
    folder_name = None
    file_name = None
    for fld in log_folders:
        tmp_name = [fn for fn in os.listdir(fld) if split_name in fn]
        assert len(tmp_name) <= 1, 'ERROR: Multiple files found in current folder!'
        if len(tmp_name) == 1:  # Match
            assert folder_name is None, 'ERROR: Folder not unique!'
            assert file_name is None, 'ERROR: File not unique!'
            folder_name = fld
            file_name = tmp_name[0]
    assert folder_name is not None and file_name is not None, f'ERROR: Log file ..{split_name} not found!'

    # Read log file
    file_name = file_name.replace("Indices", "Stats")  # In any case, the data log to load is the ..Stats.. one, if existing
    dlog = os.path.join(folder_name, file_name)
    if not os.path.exists(dlog):
        continue
    split_count += 1
    stats_dict = np.load(dlog)
    conn_count_orig_sel += np.sum(stats_dict['input_conn_count_sel'])
    conn_count_pred_sel += np.sum(stats_dict['output_conn_count_sel_avg'])
    conn_count_orig_all += np.sum(stats_dict.get('input_conn_count', 0))
    conn_count_pred_all += np.sum(stats_dict['output_conn_count_sel_avg']) + np.sum(stats_dict.get('input_conn_count', 0)) - np.sum(stats_dict['input_conn_count_sel'])
    if log_type == 'Rewiring':
        conn_count_actu_sel += np.sum(stats_dict['output_conn_count_sel'])
        assert stats_dict['output_conn_count'] == np.sum(stats_dict['output_conn_count_sel']) + np.sum(stats_dict['input_conn_count']) - np.sum(stats_dict['input_conn_count_sel'])
        conn_count_actu_all += stats_dict['output_conn_count']
        diff_from_avg += list(stats_dict['output_conn_count_sel'] - stats_dict['output_conn_count_sel_avg'])

print(f'Non-empty splits: {split_count}/{N_split}\n')

if log_type == 'Rewiring':
    print('<ACTUAL REWIRED INSTANCE>')
    print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_rewired = {conn_count_actu_all}, DIFF = {conn_count_actu_all - conn_count_orig_all} ({100.0 * (conn_count_actu_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
    print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_rewired = {conn_count_actu_sel}, DIFF = {conn_count_actu_sel - conn_count_orig_sel} ({100.0 * (conn_count_actu_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')
    print(f'Deviation from average prediction of ingoing #conns of {len(diff_from_avg)} post-neurons: MIN = {np.min(diff_from_avg)}, MAX = {np.max(diff_from_avg)}, #MISMATCH = {np.sum(np.array(diff_from_avg) != 0)}, OVERALL #Conn DIFF = {np.sum(diff_from_avg)}')
    print()

p_scale = conn_count_orig_sel / conn_count_pred_sel
print('<AVERAGE PREDICTION>')
print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_predicted = {conn_count_pred_all}, DIFF = {conn_count_pred_all - conn_count_orig_all} ({100.0 * (conn_count_pred_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_predicted = {conn_count_pred_sel}, DIFF = {conn_count_pred_sel - conn_count_orig_sel} ({100.0 * (conn_count_pred_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')
print(f'Probability scaling p_scale = {conn_count_orig_sel} / {conn_count_pred_sel} = {p_scale}')

Non-empty splits: 454/500

<ACTUAL REWIRED INSTANCE>
Overall: #Conns_orig = 82691527, #Conns_rewired = 82691527, DIFF = 0 (0.00%)
Within wiring target: #Conns_orig = 7203362, #Conns_rewired = 7203362, DIFF = 0 (0.00%)
Deviation from average prediction of ingoing #conns of 26787 post-neurons: MIN = 0, MAX = 0, #MISMATCH = 0, OVERALL #Conn DIFF = 0

<AVERAGE PREDICTION>
Overall: #Conns_orig = 82691527, #Conns_predicted = 82691527, DIFF = 0 (0.00%)
Within wiring target: #Conns_orig = 7203362, #Conns_predicted = 7203362, DIFF = 0 (0.00%)
Probability scaling p_scale = 7203362 / 7203362 = 1.0


---
After successful rewiring: The following section contains a manual collection of connection and synapse counts from the final log files

---

In [86]:
# Manually collecting synapse counts from log files and computing percentages
syn_orig_tot = 407127134
syn_orig_h0 = 31212240
syn_rew_tot = [402518201, 405187752, 406088376, 406247297, 407068355]
syn_diff = [-4608933, -1939382, -1038758, -879837, -58779]
for idx, (cnt, diff) in enumerate(zip(syn_rew_tot, syn_diff)):
    pct_tot = 100.0 * (cnt - syn_orig_tot) / syn_orig_tot
    syn_rew_h0 = syn_orig_h0 + diff
    pct_h0 = 100.0 * (syn_rew_h0 - syn_orig_h0) / syn_orig_h0
    print(f"Order {idx + 1}:")
    print(f"  Diff total: {pct_tot:.2f}% ({cnt} of {syn_orig_tot})")
    print(f"  Diff hex0EE: {pct_h0:.2f}% ({syn_rew_h0} of {syn_orig_h0})")

Order 1:
  Diff total: -1.13% (402518201 of 407127134)
  Diff hex0EE: -14.77% (26603307 of 31212240)
Order 2:
  Diff total: -0.48% (405187752 of 407127134)
  Diff hex0EE: -6.21% (29272858 of 31212240)
Order 3:
  Diff total: -0.26% (406088376 of 407127134)
  Diff hex0EE: -3.33% (30173482 of 31212240)
Order 4:
  Diff total: -0.22% (406247297 of 407127134)
  Diff hex0EE: -2.82% (30332403 of 31212240)
Order 5:
  Diff total: -0.01% (407068355 of 407127134)
  Diff hex0EE: -0.19% (31153461 of 31212240)


### Results summary table

|Connectome|&#124;|Total conns|Diff|Pct   |&#124;|Total syns|Diff    |Pct   |&#124;|Hex0EE conns|Diff|Pct   |&#124;|Hex0EE syns|Diff    |Pct    |&#124;|Calib runs|Opt $p_{scale}$   |
|----------|:----:|----------:|---:|-----:|:----:|---------:|-------:|-----:|:----:|-----------:|---:|-----:|:----:|----------:|-------:|------:|:----:|----------|------------------|
|Original  |&#124;|82691527   |-   |-     |&#124;|407127134 |-       |-     |&#124;|7203362     |-   |-     |&#124;|31212240   |-       |-      |&#124;|-         |-                 |
|1st order |&#124;|82693868   |2341| 0.00%|&#124;|402518201 |-4608933|-1.13%|&#124;|7205703     |2341|0.03% |&#124;|26603307   |-4608933|-14.77%|&#124;|7 (osc)   |1.0               |
|2nd order |&#124;|82691526   |-1  |-0.00%|&#124;|405187752 |-1939382|-0.48%|&#124;|7203361     |-1  |-0.00%|&#124;|29272858   |-1939382|-6.21% |&#124;|8 (osc)   |1.0044424693787228|
|3rd order |&#124;|82691529   |2   | 0.00%|&#124;|406088376 |-1038758|-0.26%|&#124;|7203364     | 2  | 0.00%|&#124;|30173482   |-1038758|-3.33% |&#124;|5 (osc)   |1.004630412866105 |
|4th order |&#124;|82691527   |0   | 0.00%|&#124;|406247297 |-879837 |-0.22%|&#124;|7203362     | 0  | 0.00%|&#124;|30332403   |-879837 |-2.82% |&#124;|6 (conv)  |1.0207973895957692|
|5th order |&#124;|82691527   |0   | 0.00%|&#124;|407068355 |-58779  |-0.01%|&#124;|7203362     | 0  | 0.00%|&#124;|31153461   |-58779  |-0.19% |&#124;|3 (conv)  |1.0195503473866605|